### CICModbus2023 Flow labeling 

In [3]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import glob
import os
from sys import platform
import datetime


class ModbusDataset():
    """
    A class to organize and manage the CICModbus 2023 directories with simple hierarchical structure
    and metadata.
    """
    def __init__(self, _root_dir = "./ModbusDataset"):
        self.root_dir = _root_dir
        datasets_dir = self.find_csv_in_folder(root_dir,"output")
        benign_datasets_dir = self.find_csv_in_list(datasets_dir,"benign")
        attack_dataset_dir = self.find_csv_in_list(datasets_dir,"attack")
        ext_attack_dataset_dir = self.find_csv_in_list(attack_dataset_dir,"external")
        comp_ied_attack_dataset_dir = self.find_csv_in_list(attack_dataset_dir,"compromised-ied")
        comp_scada_attack_dataset_dir =self.find_csv_in_list(attack_dataset_dir,"compromised-scada")
        attack_logs_dir = self.find_csv_in_folder(root_dir,"attack logs")
        attack_logs_dir.extend(self.find_csv_in_folder(root_dir,"attacker logs"))
        ## Corrupted TimeStamp (more detail in ./ModbusDataset/Readme.md)
        attack_logs_dir.remove("./ModbusDataset/attack/compromised-scada/attack logs/03-21-2023/03-21-2023-1-original.csv")
        ext_attack_log_dir = self.find_csv_in_list(attack_logs_dir,"external")
        comp_ied_attack_log_dir = self.find_csv_in_list(attack_logs_dir,"compromised-ied")
        comp_scada_attack_log_dir =self.find_csv_in_list(attack_logs_dir,"compromised-scada")


        self.dataset ={
            "metadata":{
                "total_dataset_num":len(datasets_dir),"benign_dataset_num":len(benign_datasets_dir),"attack_dataset_num":{
                "total_num":len(attack_dataset_dir),
                "external_num":len(ext_attack_dataset_dir),
                "compromised-ied_num":len(comp_ied_attack_dataset_dir),
                "compromised-scada_num":len(comp_scada_attack_dataset_dir),
                },"attack_logs_num":{
                    "total_num":len(attack_logs_dir),
                    "external_num":(ext_attack_log_dir),
                    "compromised-ied_num":len(comp_ied_attack_log_dir),
                    "compromised-scada_num":len(comp_scada_attack_log_dir),
                }
            },
            "benign_dataset_dir":benign_datasets_dir,
            "attack_dataset_dir":{"total":attack_dataset_dir,
                                  "external":ext_attack_dataset_dir,
                                  "compromised-ied":comp_ied_attack_dataset_dir,
                                  "compromised-scada":comp_scada_attack_dataset_dir}
            ,
            "attack_log_dir":{"total":attack_logs_dir,
                            "external":ext_attack_log_dir,
                            "compromised-ied":comp_ied_attack_log_dir,
                            "compromised-scada":comp_scada_attack_log_dir}}
        
    def find_csv_in_folder(self,_start_path,_folder_name):
        csv_files = []
        for root, _, files in os.walk(_start_path):
            if _folder_name in root.split(os.sep) :
                csv_files.extend([os.path.join(root, f) for f in files if f.endswith('.csv')])
        return csv_files

    def find_csv_in_list(self,_datasets_dir,_folder_name):
        return  [ds for ds in _datasets_dir  if ds.find(_folder_name)!=(-1)]
    
    def summary_print(self):
        print("num of total fix-ordered dataset files ",self.dataset["metadata"]["total_dataset_num"])
        print("num of benign dataset files",self.dataset["metadata"]["benign_dataset_num"])
        print("num of attack dataset files",self.dataset["metadata"]["attack_dataset_num"]["total_num"])
        print("num of attack logs ",self.dataset["metadata"]["attack_logs_num"]["total_num"])
    
    def print_csv(datasets_dir_list,replace_dir):
        for i,dataset in enumerate(datasets_dir_list) :
            print(i+1,dataset.replace(replace_dir,""))

print("pandas current version ",pd.__version__)
print("numpy current version ",np.__version__)
root_dir = "./ModbusDataset"
modbus = ModbusDataset(root_dir)
modbus.summary_print()


pandas current version  2.2.0
numpy current version  1.26.3
num of total fix-ordered dataset files  170
num of benign dataset files 62
num of attack dataset files 108
num of attack logs  34


/tmp/ipykernel_151614/3015332771.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:

print("list of benign csv dataset directories under ./ModbusDataset/benign/*")
ModbusDataset.print_csv(modbus.dataset["benign_dataset_dir"],"./ModbusDataset/benign/")
print("list of attack csv dataset directories under ./ModbusDataset/attack/*")
ModbusDataset.print_csv(modbus.dataset["attack_dataset_dir"]["total"],"./ModbusDataset/attack/")


list of benign csv dataset directories under ./ModbusDataset/benign/*
1 network-wide-pcap-capture/network-wide/output/network-wide-normal-18_fix_ord.pcap_Flow.csv
2 network-wide-pcap-capture/network-wide/output/network-wide-normal-22_fix_ord.pcap_Flow.csv
3 network-wide-pcap-capture/network-wide/output/network-wide-normal-30_fix_ord.pcap_Flow.csv
4 network-wide-pcap-capture/network-wide/output/network-wide-normal-19_fix_ord.pcap_Flow.csv
5 network-wide-pcap-capture/network-wide/output/network-wide-normal-32_fix_ord.pcap_Flow.csv
6 network-wide-pcap-capture/network-wide/output/network-wide-normal-25_fix_ord.pcap_Flow.csv
7 network-wide-pcap-capture/network-wide/output/network-wide-normal-15_fix_ord.pcap_Flow.csv
8 network-wide-pcap-capture/network-wide/output/network-wide-normal-31_fix_ord.pcap_Flow.csv
9 network-wide-pcap-capture/network-wide/output/network-wide-normal-24_fix_ord.pcap_Flow.csv
10 network-wide-pcap-capture/network-wide/output/network-wide-normal-14_fix_ord.pcap_Flow.csv

#### Labeling functions


In [45]:
# Basic preprocessing before getting started on labelling.
# Deletes rows with "Infinity" and NaNs, converts "Timestamp" to Pandas Datetime, and converts all necessary columns to numeric values
# Int_64 Columns (Attempted-Category) not considered.

print_index = False

def format_csv_for_labelling(df):
    df = df.replace('Infinity', np.nan)
    # Clean the Timestamp strings to always include microseconds (append .0 if missing)
    df['Timestamp'] = pd.to_datetime(
        df['Timestamp'].apply(
            lambda x: x if '.' in x.split()[-1] else f"{x}.0"  # Split into date/time and check time part
        ),
        format='%Y-%m-%d %H:%M:%S.%f'  # Parse with microseconds
    )
    for column in df.columns:
        if column not in ['Flow ID' , 'Timestamp', 'Src IP', 'Dst IP', 'Label','Attack','TransactionID','TargetIP']:
            df[column] = pd.to_numeric(df[column], errors='coerce')
    df.dropna()
    return df.dropna()

def read_csvs_from_path_and_reformat(path):
    df = pd.read_csv(path, encoding='cp1252')

    df = format_csv_for_labelling(df)
    print("labels after pre-processing:\n", df["Label"].value_counts())

    int32_columns = ["Src Port", "Dst Port", "Flow Duration", "Total Fwd Packet", "Total Bwd packets", "Total Length of Fwd Packet", "Total Length of Bwd Packet", "Fwd Packet Length Max",
        "Fwd Packet Length Min", "Bwd Packet Length Max", "Bwd Packet Length Min", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Total",
        "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags", "Fwd URG Flags", "Bwd URG Flags", "Packet Length Min", "Packet Length Max", "FIN Flag Count", "SYN Flag Count", "RST Flag Count", "PSH Flag Count",
        "ACK Flag Count", "URG Flag Count", "CWR Flag Count", "ECE Flag Count", "Subflow Fwd Packets", "Subflow Fwd Bytes",
        "Subflow Bwd Packets", "Subflow Bwd Bytes", "FWD Init Win Bytes", "Bwd Init Win Bytes", "Fwd Act Data Pkts", "Fwd Seg Size Min", "Active Max",
        "Active Min", "Idle Max", "Idle Min"]

    int16_columns = ["Fwd Header Length", "Bwd Header Length", "ICMP Code", "ICMP Type"]

    for column in int32_columns:
        df[column] = df[column].astype('int32')

    for column in int16_columns:
        df[column] = df[column].astype('int16')

    return df

# Main labelling function. Only used for labelling Malicious flows.
# Timestamps are in NANOSECONDS (!) Unix time. Note that the CSV files are in the UTC timezone.
# df = dataframe with flows. Note that labelling happens inplace on the 'df' parameter, and so this function doesn't return anything
# label = the label that will be given to flows matching the criteria specified in the function
# additional_filters = add any additional constraints that cannot be covered by the already provided function arguments
# payload_filter = When set to true, this will automatically add a constraint ["Total Length of Fwd Packet"] == 0. Note that

def label_flows(df, label, attack_start_time_nanoseconds, attack_end_time_nanoseconds, src_ip_list=None,
                dst_ip_list= None, src_port_list=None, dst_port_list=None, additional_filters=[], attempted_category=-1, payload_filter=False):

    # Create initial mask for whole df with all values set to True. Squeeze is necessary to remove second axis (with value 1)
    # The reason is that a df of shape (X,) gets converted to (1,X) if you '&' it with a df of shape (X,1)
    mask = pd.DataFrame(True,index=df.index,columns=[df.columns[0]]).squeeze()

    attack_start_datetime = pd.to_datetime(attack_start_time_nanoseconds, unit='ns')
    attack_end_datetime = pd.to_datetime(attack_end_time_nanoseconds, unit='ns')

    mask &= (df["Timestamp"] >= attack_start_datetime)
    mask &= (df["Timestamp"] <= attack_end_datetime)

    if src_ip_list is not None:
        mask &= (df["Src IP"].isin(src_ip_list))
    if dst_ip_list is not None:
        mask &= (df["Dst IP"].isin(dst_ip_list))

    if src_port_list is not None:
        mask &= (df["Src Port"].isin(src_port_list))
    if dst_port_list is not None:
        mask &= (df["Dst Port"].isin(dst_port_list))

    if payload_filter:
        mask &= (df["Total Length of Fwd Packet"] == 0)

    for filter in additional_filters:
        mask &= filter

    df["Label"]=df["Label"].mask(mask, label)

# This function is called when all labelling of malicious flows is completed. Anything that has not yet received a label
# so far is labelled as Benign.
def label_rest_as_benign_and_write_csv(df, input_path):
    
    df["Label"]=df["Label"].mask(df["Label"] == "NeedManualLabel", "BENIGN")
    print("label count after labelling:\r\n", df["Label"].value_counts())
    # Construct the ready directory path alongside output
    ready_dir = os.path.join(os.path.dirname(os.path.dirname(input_path)), 'ready')
    os.makedirs(ready_dir, exist_ok=True)
    filename = os.path.basename(dir)
    name, ext = os.path.splitext(filename)  # Split the extension (name, ".csv")
    cleaned_name = name.replace("_fix_ord.pcap_Flow", "-labeled")
    # Construct full output path
    output_path = os.path.join(ready_dir, f"{cleaned_name}{ext}")  # Append extension
    # Adds line numbers in the first column if print_index is set to true
    if print_index:
        df.reset_index(inplace=True, drop=True)
        df.index += 1
        df.index.name = 'id'
        df.to_csv(output_path)
    else:
        df.to_csv(output_path, index=False)

def sort_merge_dataset(_dataset_dir):
    # take two columns (attack logs ) or four columns (attacker logs)
    new_df = format_csv_for_labelling(pd.read_csv(_dataset_dir[0], encoding='cp1252'))
    for _dir in _dataset_dir[1:] :
        new_df = pd.concat([format_csv_for_labelling(pd.read_csv(_dir, encoding='cp1252')),new_df],join="inner")

    # sort the merged dataframe vertically with respect to TimeStamps
    new_df_sorted = new_df.sort_values(by='Timestamp', ascending=True)
    new_df_sorted = new_df_sorted.reset_index(drop=True)
    return new_df_sorted
    

In [ ]:
# # create new csv files with new labels in ready folder (run it once)
# for dir in modbus.dataset["benign_dataset_dir"]:
#     print(dir)
#     label_rest_as_benign_and_write_csv(read_csvs_from_path_and_reformat(dir) ,dir)
            

In [44]:

#convert and merge csv attack log files into dataframe

ext_attack_log_df = sort_merge_dataset(modbus.dataset["attack_log_dir"]["external"])
compied_attack_log_df = sort_merge_dataset(modbus.dataset["attack_log_dir"]["compromised-ied"])
compscd_ext_attack_log_df = sort_merge_dataset(modbus.dataset["attack_log_dir"]["compromised-scada"])



print("external attack log \n",ext_attack_log_df.head(5),ext_attack_log_df.info())
print("compromised-ied attack log \n",compied_attack_log_df.head(5),compied_attack_log_df.info())
print("compromised-scada attack log \n",compscd_ext_attack_log_df.head(5),compscd_ext_attack_log_df.info())



---------                       Timestamp                                     Attack
0       2023-02-01 03:51:37.648                        Recon. Range: 65535
1       2023-02-01 03:51:37.696                            Recon. Complete
2       2023-02-01 03:52:46.776                        Recon. Range: 65535
3       2023-02-01 03:52:46.776                            Recon. Complete
4       2023-02-01 03:58:25.357                        Recon. Range: 65535
...                         ...                                        ...
131182  2023-02-01 13:21:52.286         Brute force or specific - Complete
131183  2023-02-01 13:21:52.286   Brute force or specific coil. Address: 0
131184  2023-02-01 13:21:52.287         Brute force or specific - Complete
131185  2023-02-01 13:22:13.415  Brute force or specific coil. Address: 13
131186  2023-02-01 13:22:13.415         Brute force or specific - Complete

[131187 rows x 2 columns]
---------                    Timestamp     TargetIP  \
0    202

#### External Attacks :
- Attacks from an unknown IP address (185.175.0.7) targeting 185.175.0.4.
#### Compromised SCADA Attacks :
- Attacks from 185.175.0.3 targeting 185.175.0.4 and 185.175.0.8.
- Days 12 to 15 missed in attack logs.
#### Compromised IED Attacks
- Attacks from 185.175.0.5 targeting 185.175.0.2.
